# M10: Test Prediction with Gemini-Augmented Data

This notebook:
1. Loads your trained KB-NER model from `models/kb_ner_best.pt`
2. Uses **Gemini-augmented test data** (already has context!)
3. Generates predictions
4. Saves predictions

**Advantage**: No Wikipedia retrieval needed - Gemini already provided context!

**Expected F1**: ~82-83% (based on validation score of 82.6%)

## Step 1: Install Packages

In [ ]:
# Install required packages
!pip install --upgrade transformers huggingface-hub
!pip install pytorch-crf

print("✓ All packages installed")

## Step 2: Imports

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizerFast, XLMRobertaModel
from torchcrf import CRF
import json
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Load Saved Model Checkpoint

In [ ]:
# Load checkpoint
checkpoint_path = 'models/kb_ner_best.pt'

print(f"Loading checkpoint from {checkpoint_path}...")
checkpoint = torch.load(checkpoint_path, map_location=device)

# Extract configuration and mappings
CONFIG = checkpoint['config']
tag2id = checkpoint['tag2id']
id2tag = checkpoint['id2tag']
num_tags = len(tag2id)

print(f"\n✓ Checkpoint loaded successfully!")
print(f"  Model trained for: {checkpoint['epoch'] + 1} epochs")
if 'val_f1' in checkpoint:
    print(f"  Validation F1: {checkpoint['val_f1']:.4f} ({checkpoint['val_f1']*100:.2f}%)")
    print(f"  Validation Precision: {checkpoint['val_precision']:.4f}")
    print(f"  Validation Recall: {checkpoint['val_recall']:.4f}")

print(f"\n  Model configuration:")
print(f"    Model: {CONFIG['model_name']}")
print(f"    Max length: {CONFIG['max_length']}")
print(f"    Number of entity types: {num_tags}")

## Step 4: Define Model Classes

In [ ]:
class KB_NER_Dataset(Dataset):
    """Dataset with knowledge-augmented inputs."""
    
    def __init__(self, data_file, tokenizer, max_length=128, tag2id=None):
        self.data = []
        with open(data_file, 'r') as f:
            for line in f:
                self.data.append(json.loads(line))
        
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tag2id = tag2id
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data[idx]
        tokens = example['tokens']
        context = example.get('context', '')
        
        sentence_encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            add_special_tokens=False,
            truncation=False,
            return_tensors=None
        )
        
        if context:
            context_encoding = self.tokenizer(
                context,
                is_split_into_words=False,
                add_special_tokens=False,
                truncation=True,
                max_length=50,
                return_tensors=None
            )
            
            input_ids = (
                [self.tokenizer.cls_token_id] + 
                sentence_encoding['input_ids'] + 
                [self.tokenizer.sep_token_id] + 
                context_encoding['input_ids'] + 
                [self.tokenizer.sep_token_id]
            )
        else:
            input_ids = (
                [self.tokenizer.cls_token_id] + 
                sentence_encoding['input_ids'] + 
                [self.tokenizer.sep_token_id]
            )
        
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length]
        
        attention_mask = [1] * len(input_ids)
        padding_length = self.max_length - len(input_ids)
        input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        
        word_ids_sentence = sentence_encoding.word_ids()
        word_ids = [None]
        word_ids.extend(word_ids_sentence)
        word_ids.append(None)
        
        if context:
            context_length = len(context_encoding['input_ids']) + 1
            word_ids.extend([None] * context_length)
        
        word_ids.extend([None] * padding_length)
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'word_ids': word_ids,
            'original_tokens': tokens,
            'example_id': example['id']
        }

print("✓ KB_NER_Dataset class defined")

In [ ]:
class XLMRobertaCRF(nn.Module):
    """XLM-RoBERTa + CRF for NER."""
    
    def __init__(self, model_name, num_tags, dropout=0.1):
        super().__init__()
        self.xlmr = XLMRobertaModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.xlmr.config.hidden_size, num_tags)
        self.crf = CRF(num_tags, batch_first=True)
    
    def forward(self, input_ids, attention_mask, labels=None, word_mask=None):
        outputs = self.xlmr(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        
        emissions = self.classifier(sequence_output)
        
        if labels is not None:
            mask = (labels != -100).byte()
            mask[:, 0] = 1
            
            labels_for_crf = labels.clone()
            labels_for_crf[labels_for_crf == -100] = 0
            
            log_likelihood = self.crf(emissions, labels_for_crf, mask=mask, reduction='mean')
            loss = -log_likelihood
            
            return loss, emissions
        else:
            if word_mask is not None:
                mask = word_mask.byte()
            else:
                mask = attention_mask.byte()
            
            predictions = self.crf.decode(emissions, mask=mask)
            return predictions

print("✓ XLMRobertaCRF model defined")

## Step 5: Load Model and Prepare Data

In [ ]:
# Initialize model
model = XLMRobertaCRF(
    CONFIG['model_name'],
    num_tags,
    dropout=CONFIG['dropout']
).to(device)

# Load trained weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

print(f"✓ Model loaded and ready for prediction")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

In [ ]:
# Custom collate function
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    word_ids = [item['word_ids'] for item in batch]
    original_tokens = [item['original_tokens'] for item in batch]
    example_ids = [item['example_id'] for item in batch]
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'word_ids': word_ids,
        'original_tokens': original_tokens,
        'example_ids': example_ids
    }

# Load test dataset (Gemini-augmented!)
tokenizer = XLMRobertaTokenizerFast.from_pretrained(CONFIG['model_name'])

test_dataset = KB_NER_Dataset(
    'test_data_gemini.jsonl',  # Using Gemini-augmented data!
    tokenizer,
    max_length=CONFIG['max_length'],
    tag2id=tag2id
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    collate_fn=collate_fn
)

print(f"✓ Test dataset loaded (Gemini-augmented)")
print(f"  Test examples: {len(test_dataset)}")

## Step 6: Generate Predictions

In [ ]:
def predict_test_data(model, dataloader, device, id2tag):
    """
    Generate predictions for test data.
    Returns predictions in the format: [(id, tokens, predicted_tags), ...]
    """
    model.eval()
    
    all_predictions = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            word_ids_batch = batch['word_ids']
            
            # Create word_mask for CRF decoding
            word_mask = torch.zeros_like(attention_mask)
            for i, word_ids in enumerate(word_ids_batch):
                for j, word_id in enumerate(word_ids):
                    if word_id is not None:
                        word_mask[i, j] = 1
            
            # CRF requires first position to be valid
            word_mask[:, 0] = 1
            word_mask = word_mask.to(device)
            
            # Get predictions
            predictions = model(input_ids, attention_mask, word_mask=word_mask)
            
            # Convert to tags
            for pred, word_ids, orig_tokens, example_id in zip(
                predictions,
                batch['word_ids'],
                batch['original_tokens'],
                batch['example_ids']
            ):
                # Align predictions with original tokens
                token_preds = []
                prev_word_idx = None
                
                for pred_id, word_idx in zip(pred, word_ids):
                    if word_idx is not None and word_idx != prev_word_idx:
                        if word_idx < len(orig_tokens):
                            token_preds.append(id2tag[pred_id])
                    prev_word_idx = word_idx
                
                # Ensure same length
                if len(token_preds) != len(orig_tokens):
                    token_preds = token_preds[:len(orig_tokens)]
                    if len(token_preds) < len(orig_tokens):
                        token_preds.extend(['O'] * (len(orig_tokens) - len(token_preds)))
                
                all_predictions.append({
                    'id': example_id,
                    'tokens': list(orig_tokens),
                    'ner_tags': token_preds
                })
    
    return all_predictions

print("✓ Prediction function defined")

In [ ]:
# Generate predictions
print("\nGenerating predictions on Gemini-augmented test data...")
predictions = predict_test_data(model, test_loader, device, id2tag)

print(f"\n✓ Predictions generated for {len(predictions)} examples")

## Step 7: Save Predictions

In [ ]:
# Save predictions in .jsonl format
output_file = 'test_predictions_m10_gemini.jsonl'

with open(output_file, 'w') as f:
    for pred in predictions:
        f.write(json.dumps(pred) + '\n')

print(f"✓ Predictions saved to {output_file}")
print(f"  Total predictions: {len(predictions)}")

# Show sample predictions
print("\n📝 Sample predictions:")
for i in range(min(5, len(predictions))):
    pred = predictions[i]
    print(f"\nExample {i+1} (ID: {pred['id']}):")
    print(f"  Tokens: {' '.join(pred['tokens'][:10])}..." if len(pred['tokens']) > 10 else f"  Tokens: {' '.join(pred['tokens'])}")
    print(f"  Tags:   {' '.join(pred['ner_tags'][:10])}..." if len(pred['ner_tags']) > 10 else f"  Tags:   {' '.join(pred['ner_tags'])}")
    
    # Count entities
    num_entities = sum(1 for tag in pred['ner_tags'] if tag.startswith('B-'))
    print(f"  Entities found: {num_entities}")

## Summary

✅ **Complete!** Your test predictions are saved to `test_predictions_m10_gemini.jsonl`

### What was done:
1. Loaded trained KB-NER model with **82.6% validation F1**
2. Used **Gemini-augmented test data** (context already provided by Gemini!)
3. Generated predictions using KB-NER model
4. Saved predictions in `.jsonl` format

### Advantages of Gemini augmentation:
- ✅ No Wikipedia API rate limits
- ✅ High-quality context from Gemini
- ✅ Fast - no retrieval delays
- ✅ Consistent context quality

### Expected performance:
- **~82-83% F1** on test set (similar to validation)
- Should beat your friends' 77-79% BERT scores! 🎉

### Files created:
- `test_predictions_m10_gemini.jsonl` - Final predictions using Gemini context

### Next steps:
Submit `test_predictions_m10_gemini.jsonl` to your evaluation system!